In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data = pd.read_excel('C:/Users/Administrator/paper/108_recruit_train.xlsx')
data = data[data['highest_school_level']!='F']
data = data.drop(['interview','second_interview','relabel'], axis=1)
data.head()

,id,sex,apply_age,marriage,highest_education,highest_graduation,loan,loan_status,military_status,aboriginal,height,weight,nationality,highest_school_level,english_level,hired_or_not,relabel_check
0,R224211691,F,24,0,B,Y,0,1,1,0,162,60,1,3,2.0,N,N
1,R223936966,F,28,0,B,Y,1,1,1,0,158,52,1,2,2.0,N,N
2,F228161684,F,24,0,B,Y,1,1,1,0,164,70,1,3,2.0,N,N
3,O200443001,F,21,0,B,Y,0,1,1,0,152,41,1,5,2.0,N,N
4,N225966272,F,22,0,B,I,0,1,1,0,155,45,1,6,3.0,N,N


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1188 entries, 0 to 1201
Data columns (total 17 columns):
id                      1188 non-null object
sex                     1188 non-null object
apply_age               1188 non-null int64
marriage                1188 non-null int64
highest_education       1188 non-null object
highest_graduation      1188 non-null object
loan                    1188 non-null int64
loan_status             1188 non-null int64
military_status         1188 non-null int64
aboriginal              1188 non-null int64
height                  1188 non-null int64
weight                  1188 non-null int64
nationality             1188 non-null int64
highest_school_level    1188 non-null object
english_level           1188 non-null float64
hired_or_not            1188 non-null object
relabel_check           1188 non-null object
dtypes: float64(1), int64(9), object(7)
memory usage: 167.1+ KB


In [4]:
print("應徵總人數", len(data))
print("男性應徵人數", len(data[data['sex']=='M']))
print("男性錄取人數", len(data[(data['sex']=='M') & (data['hired_or_not']=='Y')]))
print("男性錄取率", len(data[(data['sex']=='M') & (data['hired_or_not']=='Y')])/len(data[data['sex']=='M']))
print("女性應徵人數", len(data[data['sex']=='F']))
print("女性錄取人數", len(data[(data['sex']=='F') & (data['hired_or_not']=='Y')]))
print("女性錄取率", len(data[(data['sex']=='F') & (data['hired_or_not']=='Y')])/len(data[data['sex']=='F']))
print("男性應徵人數占比",len(data[data['sex']=='M'])/len(data))
print("女性應徵人數占比",len(data[data['sex']=='F'])/len(data))

應徵總人數 1188
男性應徵人數 412
男性錄取人數 39
男性錄取率 0.09466019417475728
女性應徵人數 776
女性錄取人數 63
女性錄取率 0.08118556701030928
男性應徵人數占比 0.3468013468013468
女性應徵人數占比 0.6531986531986532


In [5]:
# 設定資料集男女比及應徵錄取比
n_pop = 1000000
p_male = 0.3468
p_job_male, p_job_female = 0.0946, 0.0812

gender = np.random.choice(['male', 'female'], size=n_pop, p=[p_male, 1 - p_male])
df = pd.DataFrame({'gender' : gender})
df.loc[df.gender == 'male', 'found_job'] = np.random.choice([1, 0], size=len(df.loc[df.gender == 'male']), p=[p_job_male, 1 - p_job_male])
df.loc[df.gender == 'female', 'found_job'] = np.random.choice([1, 0], size=len(df.loc[df.gender == 'female']), p=[p_job_female, 1 - p_job_female])
df['found_job'] = df['found_job'].astype(int)

In [6]:
# 檢視資料集
print("男性應徵人數", len(df[df['gender']=='male']))
print("男性錄取人數", len(df[(df['gender']=='male') & (df['found_job']==1)]))
print("男性錄取率", len(df[(df['gender']=='male') & (df['found_job']==1)])/len(df[df['gender']=='male']))
print("女性應徵人數", len(df[df['gender']=='female']))
print("女性錄取人數", len(df[(df['gender']=='female') & (df['found_job']==1)]))
print("女性錄取率", len(df[(df['gender']=='female') & (df['found_job']==1)])/len(df[df['gender']=='female']))
print("男性應徵人數占比",len(df[df['gender']=='male'])/len(df))
print("女性應徵人數占比",len(df[df['gender']=='female'])/len(df))

男性應徵人數 346486
男性錄取人數 32581
男性錄取率 0.09403265932822683
女性應徵人數 653514
女性錄取人數 52795
女性錄取率 0.08078633357510322
男性應徵人數占比 0.346486
女性應徵人數占比 0.653514


In [7]:
df.pivot_table(index='gender', columns='found_job', aggfunc='size')

found_job,0,1
gender,,
female,600719,52795
male,313905,32581


In [8]:
print("男性錄取年齡平均", data[(data['sex']=='M') & (data['hired_or_not']=='Y')].apply_age.mean())
print("男性錄取年齡標準差", data[(data['sex']=='M') & (data['hired_or_not']=='Y')].apply_age.std())
print("男性未錄取年齡平均", data[(data['sex']=='M') & (data['hired_or_not']=='N')].apply_age.mean())
print("男性未錄取年齡標準差", data[(data['sex']=='M') & (data['hired_or_not']=='N')].apply_age.std())
print("---------------------------------------------------------------------")
print("女性錄取年齡平均", data[(data['sex']=='F') & (data['hired_or_not']=='Y')].apply_age.mean())
print("女性錄取年齡標準差", data[(data['sex']=='F') & (data['hired_or_not']=='Y')].apply_age.std())
print("女性未錄取年齡平均", data[(data['sex']=='F') & (data['hired_or_not']=='N')].apply_age.mean())
print("女性未錄取年齡標準差", data[(data['sex']=='F') & (data['hired_or_not']=='N')].apply_age.std())

男性錄取年齡平均 26.53846153846154
男性錄取年齡標準差 3.3938501180048863
男性未錄取年齡平均 27.396782841823057
男性未錄取年齡標準差 4.783842693106973
---------------------------------------------------------------------
女性錄取年齡平均 25.26984126984127
女性錄取年齡標準差 2.806710560059008
女性未錄取年齡平均 26.517531556802243
女性未錄取年齡標準差 4.669111814273186


In [10]:
# 設定資料集年齡分布(待採樣)
mu_job_male, sigma_job_male = 26.5385, 3.3939
mu_no_job_male, sigma_no_job_male = 27.3968, 4.7838

mu_job_female, sigma_job_female = 25.2698, 2.8067
mu_no_job_female, sigma_no_job_female = 26.5175, 4.6691

df_male = df[df['gender']=='male']
df_female = df[df['gender']=='female']

df_male.loc[df_male.found_job == 1, 'age'] = np.floor(np.random.normal(mu_job_male, sigma_job_male, len(df_male.loc[df_male.found_job == 1])))
df_male.loc[df_male.found_job == 0, 'age'] = np.floor(np.random.normal(mu_no_job_male, sigma_no_job_male, len(df_male.loc[df_male.found_job == 0])))
df_female.loc[df_female.found_job == 1, 'age'] = np.floor(np.random.normal(mu_job_female, sigma_job_female, len(df_female.loc[df_female.found_job == 1])))
df_female.loc[df_female.found_job == 0, 'age'] = np.floor(np.random.normal(mu_no_job_female, sigma_no_job_female, len(df_female.loc[df_female.found_job == 0])))

df = df_male.append(df_female)
df['age'] = df['age'].astype(int)

In [11]:
print("男性錄取年齡平均", df[(df['gender']=='male') & (df['found_job']==1)].age.mean())
print("男性錄取年齡標準差", df[(df['gender']=='male') & (df['found_job']==1)].age.std())
print("男性未錄取年齡平均", df[(df['gender']=='male') & (df['found_job']==0)].age.mean())
print("男性未錄取年齡標準差", df[(df['gender']=='male') & (df['found_job']==0)].age.std())
print("---------------------------------------------------------------------")
print("女性錄取年齡平均", df[(df['gender']=='female') & (df['found_job']==1)].age.mean())
print("女性錄取年齡標準差", df[(df['gender']=='female') & (df['found_job']==1)].age.std())
print("女性未錄取年齡平均", df[(df['gender']=='female') & (df['found_job']==0)].age.mean())
print("女性未錄取年齡標準差", df[(df['gender']=='female') & (df['found_job']==0)].age.std())

男性錄取年齡平均 26.011724624781316
男性錄取年齡標準差 3.3963541695433483
男性未錄取年齡平均 26.90393590417483
男性未錄取年齡標準差 4.790027075204018
---------------------------------------------------------------------
女性錄取年齡平均 24.741604318590774
女性錄取年齡標準差 2.8342038403692875
女性未錄取年齡平均 26.015851005212088
女性未錄取年齡標準差 4.674223276279987


In [12]:
df.groupby('found_job').mean().round(1)

,age
found_job,
0,26.3
1,25.2


In [13]:
# 設定資料集學歷分布(待採樣)
print("學士人數", len(data[data['highest_education']=='B']))
print("碩士人數", len(data[data['highest_education']=='M']))
print("博士人數", len(data[data['highest_education']=='D']))
print("---------------------------------------------------------------------")
print("學士錄取人數", len(data[(data['highest_education']=='B') & (data['hired_or_not']=='Y')]))
print("碩士錄取人數", len(data[(data['highest_education']=='M') & (data['hired_or_not']=='Y')]))
print("博士錄取人數", len(data[(data['highest_education']=='D') & (data['hired_or_not']=='Y')]))
print("總錄取人數", len(data[data['hired_or_not']=='Y']))
print("---------------------------------------------------------------------")
print("學士錄取率", len(data[(data['highest_education']=='B') & (data['hired_or_not']=='Y')])/len(data[data['hired_or_not']=='Y']))
print("碩士錄取率", len(data[(data['highest_education']=='M') & (data['hired_or_not']=='Y')])/len(data[data['hired_or_not']=='Y']))
print("博士錄取率", len(data[(data['highest_education']=='D') & (data['hired_or_not']=='Y')])/len(data[data['hired_or_not']=='Y']))
print("---------------------------------------------------------------------")
print("學士不錄取率", len(data[(data['highest_education']=='B') & (data['hired_or_not']=='N')])/len(data[data['hired_or_not']=='N']))
print("碩士不錄取率", len(data[(data['highest_education']=='M') & (data['hired_or_not']=='N')])/len(data[data['hired_or_not']=='N']))
print("博士不錄取率", len(data[(data['highest_education']=='D') & (data['hired_or_not']=='N')])/len(data[data['hired_or_not']=='N']))

學士人數 1024
碩士人數 162
博士人數 2
---------------------------------------------------------------------
學士錄取人數 87
碩士錄取人數 15
博士錄取人數 0
總錄取人數 102
---------------------------------------------------------------------
學士錄取率 0.8529411764705882
碩士錄取率 0.14705882352941177
博士錄取率 0.0
---------------------------------------------------------------------
學士不錄取率 0.8627992633517495
碩士不錄取率 0.13535911602209943
博士不錄取率 0.001841620626151013


In [14]:
educ_class_set = ['B', 'M', 'D']
educ_class_wt_job = [0.8529, 0.1471, 0]
educ_class_wt_no_job = [0.8628, 0.1354, 0.0018]

df.loc[df.found_job == 1, 'educ_class'] = np.random.choice(educ_class_set, size=len(df.loc[df.found_job == 1]), p=educ_class_wt_job)
df.loc[df.found_job == 0, 'educ_class'] = np.random.choice(educ_class_set, size=len(df.loc[df.found_job == 0]), p=educ_class_wt_no_job)

In [15]:
# 檢視資料集學歷分布
print("學士人數", len(df[df['educ_class']=='B']))
print("碩士人數", len(df[df['educ_class']=='M']))
print("博士人數", len(df[df['educ_class']=='D']))
print("---------------------------------------------------------------------")
print("學士錄取人數", len(df[(df['educ_class']=='B') & (df['found_job']==1)]))
print("碩士錄取人數", len(df[(df['educ_class']=='M') & (df['found_job']==1)]))
print("博士錄取人數", len(df[(df['educ_class']=='D') & (df['found_job']==1)]))
print("總錄取人數", len(df[df['found_job']==1]))
print("---------------------------------------------------------------------")
print("學士錄取率", len(df[(df['educ_class']=='B') & (df['found_job']==1)])/len(df[df['found_job']==1]))
print("碩士錄取率", len(df[(df['educ_class']=='M') & (df['found_job']==1)])/len(df[df['found_job']==1]))
print("博士錄取率", len(df[(df['educ_class']=='D') & (df['found_job']==1)])/len(df[df['found_job']==1]))
print("---------------------------------------------------------------------")
print("學士不錄取率", len(df[(df['educ_class']=='B') & (df['found_job']==0)])/len(df[df['found_job']==0]))
print("碩士不錄取率", len(df[(df['educ_class']=='M') & (df['found_job']==0)])/len(df[df['found_job']==0]))
print("博士不錄取率", len(df[(df['educ_class']=='D') & (df['found_job']==0)])/len(df[df['found_job']==0]))

學士人數 861584
碩士人數 136823
博士人數 1593
---------------------------------------------------------------------
學士錄取人數 72856
碩士錄取人數 12520
博士錄取人數 0
總錄取人數 85376
---------------------------------------------------------------------
學士錄取率 0.8533545727136432
碩士錄取率 0.14664542728635682
博士錄取率 0.0
---------------------------------------------------------------------
學士不錄取率 0.8623521796935134
碩士不錄取率 0.13590612098523547
博士不錄取率 0.001741699321251137


In [16]:
# 設定婚姻狀況資料分布(待採樣)
print("女性應徵總人數", len(data[data['sex']=='F']))
print("女性已婚總人數", len(data[(data['sex']=='F')&(data['marriage']==1)]))
print("女性未婚總人數", len(data[(data['sex']=='F')&(data['marriage']==0)]))
print("已錄取女性已婚人數", len(data[(data['sex']=='F')&(data['marriage']==1)&(data['hired_or_not']=='Y')]))
print("女性錄取者已婚比例", len(data[(data['sex']=='F')&(data['marriage']==1)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='F')&(data['hired_or_not']=='Y')]))
print("未錄取女性已婚人數", len(data[(data['sex']=='F')&(data['marriage']==1)&(data['hired_or_not']=='N')]))
print("女性未錄取者已婚比例", len(data[(data['sex']=='F')&(data['marriage']==1)&(data['hired_or_not']=='N')])/len(data[(data['sex']=='F')&(data['hired_or_not']=='N')]))
print("---------------------------------------------------------------------")
print("男性應徵總人數", len(data[data['sex']=='M']))
print("男性已婚總人數", len(data[(data['sex']=='M')&(data['marriage']==1)]))
print("男性未婚總人數", len(data[(data['sex']=='M')&(data['marriage']==0)]))
print("已錄取男性已婚人數", len(data[(data['sex']=='M')&(data['marriage']==1)&(data['hired_or_not']=='Y')]))
print("男性錄取者已婚比例", len(data[(data['sex']=='M')&(data['marriage']==1)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='Y')]))
print("未錄取男性已婚人數", len(data[(data['sex']=='M')&(data['marriage']==1)&(data['hired_or_not']=='N')]))
print("男性未錄取者已婚比例", len(data[(data['sex']=='M')&(data['marriage']==1)&(data['hired_or_not']=='N')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='N')]))
print("---------------------------------------------------------------------")
print("已婚女性錄取率", len(data[(data['sex']=='F')&(data['marriage']==1)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='F')&(data['marriage']==1)]))
print("未婚女性錄取率", len(data[(data['sex']=='F')&(data['marriage']==0)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='F')&(data['marriage']==0)]))
print("已婚男性錄取率", len(data[(data['sex']=='M')&(data['marriage']==1)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='M')&(data['marriage']==1)]))
print("未婚男性錄取率", len(data[(data['sex']=='M')&(data['marriage']==0)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='M')&(data['marriage']==0)]))

女性應徵總人數 776
女性已婚總人數 95
女性未婚總人數 681
已錄取女性已婚人數 3
女性錄取者已婚比例 0.047619047619047616
未錄取女性已婚人數 92
女性未錄取者已婚比例 0.12903225806451613
---------------------------------------------------------------------
男性應徵總人數 412
男性已婚總人數 31
男性未婚總人數 381
已錄取男性已婚人數 5
男性錄取者已婚比例 0.1282051282051282
未錄取男性已婚人數 26
男性未錄取者已婚比例 0.06970509383378017
---------------------------------------------------------------------
已婚女性錄取率 0.031578947368421054
未婚女性錄取率 0.0881057268722467
已婚男性錄取率 0.16129032258064516
未婚男性錄取率 0.08923884514435695


In [17]:
# 設定男女婚姻狀況分布
# 男性錄取者已婚比例, 男性未錄取者已婚比例
p_job_male_marriage, p_no_job_male_marriage = 0.12820512, 0.06970509
# 女性錄取者已婚比例, 女性未錄取者已婚比例
p_job_female_marriage, p_no_job_female_marriage = 0.04761904, 0.12903225

df_male = df[df['gender']=='male']
df_female = df[df['gender']=='female']

df_male.loc[df_male.found_job == 1, 'marriage'] = np.random.choice([1, 0], size=len(df_male.loc[df_male.found_job == 1]), p=[p_job_male_marriage, 1 - p_job_male_marriage])
df_male.loc[df_male.found_job == 0, 'marriage'] = np.random.choice([1, 0], size=len(df_male.loc[df_male.found_job == 0]), p=[p_no_job_male_marriage, 1 - p_no_job_male_marriage])
df_female.loc[df_female.found_job == 1, 'marriage'] = np.random.choice([1, 0], size=len(df_female.loc[df_female.found_job == 1]), p=[p_job_female_marriage, 1 - p_job_female_marriage])
df_female.loc[df_female.found_job == 0, 'marriage'] = np.random.choice([1, 0], size=len(df_female.loc[df_female.found_job == 0]), p=[p_no_job_female_marriage, 1 - p_no_job_female_marriage])

df = df_male.append(df_female)

In [18]:
# 設定婚姻狀況資料分布
print("女性應徵總人數", len(df[df['gender']=='female']))
print("女性已婚總人數", len(df[(df['gender']=='female')&(df['marriage']==1)]))
print("女性未婚總人數", len(df[(df['gender']=='female')&(df['marriage']==0)]))
print("已錄取女性已婚人數", len(df[(df['gender']=='female')&(df['marriage']==1)&(df['found_job']==1)]))
print("女性錄取者已婚比例", len(df[(df['gender']=='female')&(df['marriage']==1)&(df['found_job']==1)])/len(df[(df['gender']=='female')&(df['found_job']==1)]))
print("未錄取女性已婚人數", len(df[(df['gender']=='female')&(df['marriage']==1)&(df['found_job']==0)]))
print("女性未錄取者已婚比例", len(df[(df['gender']=='female')&(df['marriage']==1)&(df['found_job']==0)])/len(df[(df['gender']=='female')&(df['found_job']==0)]))
print("---------------------------------------------------------------------")
print("男性應徵總人數", len(df[df['gender']=='male']))
print("男性已婚總人數", len(df[(df['gender']=='male')&(df['marriage']==1)]))
print("男性未婚總人數", len(df[(df['gender']=='male')&(df['marriage']==0)]))
print("已錄取男性已婚人數", len(df[(df['gender']=='male')&(df['marriage']==1)&(df['found_job']==1)]))
print("男性錄取者已婚比例", len(df[(df['gender']=='male')&(df['marriage']==1)&(df['found_job']==1)])/len(df[(df['gender']=='male')&(df['found_job']==1)]))
print("未錄取男性已婚人數", len(df[(df['gender']=='male')&(df['marriage']==1)&(df['found_job']==0)]))
print("男性未錄取者已婚比例", len(df[(df['gender']=='male')&(df['marriage']==1)&(df['found_job']==0)])/len(df[(df['gender']=='male')&(df['found_job']==0)]))
print("---------------------------------------------------------------------")
print("已婚女性錄取率", len(df[(df['gender']=='female')&(df['marriage']==1)&(df['found_job']==1)])/len(df[(df['gender']=='female')&(df['marriage']==1)]))
print("未婚女性錄取率", len(df[(df['gender']=='female')&(df['marriage']==0)&(df['found_job']==1)])/len(df[(df['gender']=='female')&(df['marriage']==0)]))
print("已婚男性錄取率", len(df[(df['gender']=='male')&(df['marriage']==1)&(df['found_job']==1)])/len(df[(df['gender']=='male')&(df['marriage']==1)]))
print("未婚男性錄取率", len(df[(df['gender']=='male')&(df['marriage']==0)&(df['found_job']==1)])/len(df[(df['gender']=='male')&(df['marriage']==0)]))

女性應徵總人數 653514
女性已婚總人數 79946
女性未婚總人數 573568
已錄取女性已婚人數 2467
女性錄取者已婚比例 0.046727909839946964
未錄取女性已婚人數 77479
女性未錄取者已婚比例 0.12897710909759805
---------------------------------------------------------------------
男性應徵總人數 346486
男性已婚總人數 25908
男性未婚總人數 320578
已錄取男性已婚人數 4104
男性錄取者已婚比例 0.12596298456155428
未錄取男性已婚人數 21804
男性未錄取者已婚比例 0.06946050556697089
---------------------------------------------------------------------
已婚女性錄取率 0.03085832937232632
未婚女性錄取率 0.08774548091943762
已婚男性錄取率 0.15840666975451598
未婚男性錄取率 0.08883017549551123


In [19]:
# 檢視資料集借貸與錄取與否關係
print("有借貸者人數", len(data[data['loan']==1]))
print("有借貸且信用正常且錄取者", len(data[(data['loan']==1)&(data['loan_status']==1)&(data['hired_or_not']=='Y')]))
print("有借貸但信用異常且錄取者", len(data[(data['loan']==1)&(data['loan_status']==0)&(data['hired_or_not']=='Y')]))
print("無借貸且錄取者", len(data[(data['loan']==0)&(data['hired_or_not']=='Y')]))
print("有借貸且異常者占有信貸者比例", len(data[(data['loan']==1)&(data['loan_status']==0)])/len(data[data['loan']==1]))
print("---------------------------------------------------------------------")
print("有信貸者錄取率", len(data[(data['loan']==1)&(data['hired_or_not']=='Y')])/len(data[data['loan']==1]))
print("無信貸者錄取率", len(data[(data['loan']==0)&(data['hired_or_not']=='Y')])/len(data[data['loan']==0]))
print("---------------------------------------------------------------------")
print("錄取者有信貸比例", len(data[(data['loan']==1)&(data['hired_or_not']=='Y')])/len(data[(data['hired_or_not']=='Y')]))
print("未錄取者有信貸比例", len(data[(data['loan']==1)&(data['hired_or_not']=='N')])/len(data[(data['hired_or_not']=='N')]))

有借貸者人數 276
有借貸且信用正常且錄取者 28
有借貸但信用異常且錄取者 0
無借貸且錄取者 74
有借貸且異常者占有信貸者比例 0.014492753623188406
---------------------------------------------------------------------
有信貸者錄取率 0.10144927536231885
無信貸者錄取率 0.08114035087719298
---------------------------------------------------------------------
錄取者有信貸比例 0.27450980392156865
未錄取者有信貸比例 0.2283609576427256


In [20]:
# 建立信貸者資料
p_job_loan, p_no_job_loan = 0.27450980, 0.22836095
df.loc[df.found_job == 1, 'loan'] = np.random.choice([1, 0], size=len(df.loc[df.found_job == 1]), p=[p_job_loan, 1 - p_job_loan])
df.loc[df.found_job == 0, 'loan'] = np.random.choice([1, 0], size=len(df.loc[df.found_job == 0]), p=[p_no_job_loan, 1 - p_no_job_loan])

In [58]:
# 檢視資料集借貸與錄取與否關係
print("有借貸者人數", len(df[df['loan']==1]))
print("無借貸且錄取者", len(df[(df['loan']==0)&(df['found_job']==1)]))
print("---------------------------------------------------------------------")
print("有信貸者錄取率", len(df[(df['loan']==1)&(df['found_job']==1)])/len(df[df['loan']==1]))
print("無信貸者錄取率", len(df[(df['loan']==0)&(df['found_job']==1)])/len(df[df['loan']==0]))
print("---------------------------------------------------------------------")
print("錄取者有信貸比例", len(df[(df['loan']==1)&(df['found_job']==1)])/len(df[(df['found_job']==1)]))
print("未錄取者有信貸比例", len(df[(df['loan']==1)&(df['found_job']==0)])/len(df[(df['found_job']==0)]))

有借貸者人數 232603
無借貸且錄取者 62010
---------------------------------------------------------------------
有信貸者錄取率 0.10045442234193024
無信貸者錄取率 0.08080563254742981
---------------------------------------------------------------------
錄取者有信貸比例 0.2736834707646177
未錄取者有信貸比例 0.22876832446994613


In [49]:
# 檢視資料集不良信貸者
print("有信貸人數", len(data[(data['loan']==1)]))
print("有信貸且信用正常人數", len(data[(data['loan']==1)&(data['loan_status']==1)]))
print("有信貸但信用不正常人數", len(data[(data['loan']==1)&(data['loan_status']==0)]))
print("---------------------------------------------------------------------")
print("有信貸且信用正常錄取人數", len(data[(data['loan']==1)&(data['loan_status']==1)&(data['hired_or_not']=='Y')]))
print("有信貸且信用正常但未錄取人數", len(data[(data['loan']==1)&(data['loan_status']==1)&(data['hired_or_not']=='N')]))
print("有信貸但信用不正常錄取人數", len(data[(data['loan']==1)&(data['loan_status']==0)&(data['hired_or_not']=='Y')]))
print("有信貸但信用不正常且未錄取人數", len(data[(data['loan']==1)&(data['loan_status']==0)&(data['hired_or_not']=='N')]))
print("---------------------------------------------------------------------")
print("未錄取者中信用正常者佔比", len(data[(data['loan']==1)&(data['loan_status']==1)&(data['hired_or_not']=='N')])/len(data[(data['loan']==1)&(data['hired_or_not']=='N')]))
print("未錄取者中信用不正常者佔比", len(data[(data['loan']==1)&(data['loan_status']==0)&(data['hired_or_not']=='N')])/len(data[(data['loan']==1)&(data['hired_or_not']=='N')]))

有信貸人數 276
有信貸且信用正常人數 272
有信貸但信用不正常人數 4
---------------------------------------------------------------------
有信貸且信用正常錄取人數 28
有信貸且信用正常但未錄取人數 244
有信貸但信用不正常錄取人數 0
有信貸但信用不正常且未錄取人數 4
---------------------------------------------------------------------
未錄取者中信用正常者佔比 0.9838709677419355
未錄取者中信用不正常者佔比 0.016129032258064516


In [55]:
# 設定信貸狀況分布
# 未錄取信用狀況良好者
p_no_job_loan_good = 0.98387096

df_loan = df[df['loan']==1]
df_no_loan = df[df['loan']==0]

df_loan.loc[df_loan.found_job == 1, 'loan_status'] = np.random.choice([1, 0], size=len(df_loan.loc[df_loan.found_job == 1]), p=[1, 0])
df_loan.loc[df_loan.found_job == 0, 'loan_status'] = np.random.choice([1, 0], size=len(df_loan.loc[df_loan.found_job == 0]), p=[p_no_job_loan_good, 1-p_no_job_loan_good])

df = df_loan.append(df_no_loan)

In [57]:
# 檢視資料集不良信貸者
print("有信貸人數", len(df[(df['loan']==1)]))
print("有信貸且信用正常人數", len(df[(df['loan']==1)&(df['loan_status']==1)]))
print("有信貸但信用不正常人數", len(df[(df['loan']==1)&(df['loan_status']==0)]))
print("---------------------------------------------------------------------")
print("有信貸且信用正常錄取人數", len(df[(df['loan']==1)&(df['loan_status']==1)&(df['found_job']==1)]))
print("有信貸且信用正常但未錄取人數", len(df[(df['loan']==1)&(df['loan_status']==1)&(df['found_job']==0)]))
print("有信貸但信用不正常錄取人數", len(df[(df['loan']==1)&(df['loan_status']==0)&(df['found_job']==1)]))
print("有信貸但信用不正常且未錄取人數", len(df[(df['loan']==1)&(df['loan_status']==0)&(df['found_job']==0)]))
print("---------------------------------------------------------------------")
print("未錄取者中信用正常者佔比", len(df[(df['loan']==1)&(df['loan_status']==1)&(df['found_job']==0)])/len(df[(df['loan']==1)&(df['found_job']==0)]))
print("未錄取者中信用不正常者佔比", len(df[(df['loan']==1)&(df['loan_status']==0)&(df['found_job']==0)])/len(df[(df['loan']==1)&(df['found_job']==0)]))

有信貸人數 232603
有信貸且信用正常人數 229269
有信貸但信用不正常人數 3334
---------------------------------------------------------------------
有信貸且信用正常錄取人數 23366
有信貸且信用正常但未錄取人數 205903
有信貸但信用不正常錄取人數 0
有信貸但信用不正常且未錄取人數 3334
---------------------------------------------------------------------
未錄取者中信用正常者佔比 0.9840659156841285
未錄取者中信用不正常者佔比 0.015934084315871476


In [70]:
# 檢視服役狀況
print("役畢總人數", len(data[(data['sex']=='M')&(data['military_status']==1)]))
print("待畢總人數", len(data[(data['sex']=='M')&(data['military_status']==0)]))
print("錄取者中役畢人數", len(data[(data['sex']=='M')&(data['military_status']==1)&(data['hired_or_not']=='Y')]))
print("錄取者中未畢人數", len(data[(data['sex']=='M')&(data['military_status']==0)&(data['hired_or_not']=='Y')]))
print("未錄取者中役畢人數", len(data[(data['sex']=='M')&(data['military_status']==1)&(data['hired_or_not']=='N')]))
print("未錄取者中未畢人數", len(data[(data['sex']=='M')&(data['military_status']==0)&(data['hired_or_not']=='N')]))
print("---------------------------------------------------------")
print("錄取者中役畢占比", len(data[(data['sex']=='M')&(data['military_status']==1)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='Y')]))
print("錄取者中未畢占比", len(data[(data['sex']=='M')&(data['military_status']==0)&(data['hired_or_not']=='Y')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='Y')]))
print("未錄取者中役畢占比", len(data[(data['sex']=='M')&(data['military_status']==1)&(data['hired_or_not']=='N')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='N')]))
print("未錄取者中未畢占比", len(data[(data['sex']=='M')&(data['military_status']==0)&(data['hired_or_not']=='N')])/len(data[(data['sex']=='M')&(data['hired_or_not']=='N')]))

役畢總人數 399
待畢總人數 13
錄取者中役畢人數 38
錄取者中未畢人數 1
未錄取者中役畢人數 361
未錄取者中未畢人數 12
---------------------------------------------------------
錄取者中役畢占比 0.9743589743589743
錄取者中未畢占比 0.02564102564102564
未錄取者中役畢占比 0.967828418230563
未錄取者中未畢占比 0.032171581769437


In [79]:
# 設定資料集服役情況
p_job_miliarty_status, p_no_job_military_status = 0.97435897, 0.96782841

df_male = df[df['gender']=='male']
df_female = df[df['gender']=='female']

df_male.loc[df_male.found_job == 1, 'military_status'] = np.random.choice([1, 0], size=len(df_male.loc[df_male.found_job == 1]), p=[p_job_miliarty_status, 1-p_job_miliarty_status])
df_male.loc[df_male.found_job == 0, 'military_status'] = np.random.choice([1, 0], size=len(df_male.loc[df_male.found_job == 0]), p=[p_no_job_military_status, 1-p_no_job_military_status])
df_female.loc[:,'military_status'] = 1

df = df_male.append(df_female)

In [84]:
# 檢視服役狀況
print("役畢總人數", len(df[(df['gender']=='male')&(df['military_status']==1)]))
print("待畢總人數", len(df[(df['gender']=='male')&(df['military_status']==0)]))
print("錄取者中役畢人數", len(df[(df['gender']=='male')&(df['military_status']==1)&(df['found_job']==1)]))
print("錄取者中未畢人數", len(df[(df['gender']=='male')&(df['military_status']==0)&(df['found_job']==1)]))
print("未錄取者中役畢人數", len(df[(df['gender']=='male')&(df['military_status']==1)&(df['found_job']==0)]))
print("未錄取者中未畢人數", len(df[(df['gender']=='male')&(df['military_status']==0)&(df['found_job']==0)]))
print("---------------------------------------------------------")
print("錄取者中役畢占比", len(df[(df['gender']=='male')&(df['military_status']==1)&(df['found_job']==1)])/len(df[(df['gender']=='male')&(df['found_job']==1)]))
print("錄取者中未畢占比", len(df[(df['gender']=='male')&(df['military_status']==0)&(df['found_job']==1)])/len(df[(df['gender']=='male')&(df['found_job']==1)]))
print("未錄取者中役畢占比", len(df[(df['gender']=='male')&(df['military_status']==1)&(df['found_job']==0)])/len(df[(df['gender']=='male')&(df['found_job']==0)]))
print("未錄取者中未畢占比", len(df[(df['gender']=='male')&(df['military_status']==0)&(df['found_job']==0)])/len(df[(df['gender']=='male')&(df['found_job']==0)]))

役畢總人數 335508
待畢總人數 10978
錄取者中役畢人數 31766
錄取者中未畢人數 815
未錄取者中役畢人數 303742
未錄取者中未畢人數 10163
---------------------------------------------------------
錄取者中役畢占比 0.9749854209508609
錄取者中未畢占比 0.02501457904913907
未錄取者中役畢占比 0.9676239626638633
未錄取者中未畢占比 0.032376037336136726


In [92]:
print("原住民身分總人數", len(data[data['aboriginal']==1]))
print("錄取者屬原住民身分者", len(data[(data['aboriginal']==1)&(data['hired_or_not']=='Y')]))
print("未錄取者屬原住民身分者", len(data[(data['aboriginal']==1)&(data['hired_or_not']=='N')]))
print("---------------------------------------------------------")
print("錄取者屬原住民身分者占比", len(data[(data['aboriginal']==1)&(data['hired_or_not']=='Y')])/len(data[(data['hired_or_not']=='Y')]))
print("未錄取者屬原住民身分者占比", len(data[(data['aboriginal']==1)&(data['hired_or_not']=='N')])/len(data[(data['hired_or_not']=='N')]))

原住民身分總人數 10
錄取者屬原住民身分者 1
未錄取者屬原住民身分者 9
---------------------------------------------------------
錄取者屬原住民身分者占比 0.00980392156862745
未錄取者屬原住民身分者占比 0.008287292817679558


In [93]:
# 設定資料集原住民身分
p_job_aboriginal, p_no_job_aboriginal = 0.00980392, 0.00828729

df.loc[df.found_job == 1, 'aboriginal'] = np.random.choice([1, 0], size=len(df.loc[df.found_job == 1]), p=[p_job_aboriginal, 1-p_job_aboriginal])
df.loc[df.found_job == 0, 'aboriginal'] = np.random.choice([1, 0], size=len(df.loc[df.found_job == 0]), p=[p_no_job_aboriginal, 1-p_no_job_aboriginal])

In [94]:
print("原住民身分總人數", len(df[df['aboriginal']==1]))
print("錄取者屬原住民身分者", len(df[(df['aboriginal']==1)&(df['found_job']==1)]))
print("未錄取者屬原住民身分者", len(df[(df['aboriginal']==1)&(df['found_job']==0)]))
print("---------------------------------------------------------")
print("錄取者屬原住民身分者占比", len(df[(df['aboriginal']==1)&(df['found_job']==1)])/len(df[(df['found_job']==1)]))
print("未錄取者屬原住民身分者占比", len(df[(df['aboriginal']==1)&(df['found_job']==0)])/len(df[(df['found_job']==0)]))

原住民身分總人數 8442
錄取者屬原住民身分者 870
未錄取者屬原住民身分者 7572
---------------------------------------------------------
錄取者屬原住民身分者占比 0.010190217391304348
未錄取者屬原住民身分者占比 0.008278811839619342


In [107]:
# 檢視資料集 BMI 分布
data.loc[:,'bmi'] = (data['weight'] / (data['height']/100 * data['height']/100)).round(1)
print("錄取男性bmi平均", data[(data['sex']=='M')&(data['hired_or_not']=='Y')].bmi.mean())
print("錄取男性bmi標準差", data[(data['sex']=='M')&(data['hired_or_not']=='Y')].bmi.std())
print("未錄取男性bmi平均", data[(data['sex']=='M')&(data['hired_or_not']=='N')].bmi.mean())
print("未錄取男性bmi標準差", data[(data['sex']=='M')&(data['hired_or_not']=='N')].bmi.std())
print("---------------------------------------------------------")
print("錄取女性bmi平均", data[(data['sex']=='F')&(data['hired_or_not']=='Y')].bmi.mean())
print("錄取女性bmi標準差", data[(data['sex']=='F')&(data['hired_or_not']=='Y')].bmi.std())
print("未錄取女性bmi平均", data[(data['sex']=='F')&(data['hired_or_not']=='N')].bmi.mean())
print("未錄取女性bmi標準差", data[(data['sex']=='F')&(data['hired_or_not']=='N')].bmi.std())

錄取男性bmi平均 23.474358974358978
錄取男性bmi標準差 2.583624336054209
未錄取男性bmi平均 22.98739946380696
未錄取男性bmi標準差 3.2864133706349055
---------------------------------------------------------
錄取女性bmi平均 19.603174603174608
錄取女性bmi標準差 1.7807182543232611
未錄取女性bmi平均 20.42272089761572
未錄取女性bmi標準差 2.564294567958626


In [ ]:
# 設定資料集BMI分布
mu_job_male_bmi, mu_job_female_bmi = 23.47435897, 19.60317460
sigma_job_male_bmi, sigma_job_female_bmi = 2.58362433, 1.78071825
mu_no_job_male_bmi, mu_no_job_female_bmi = 22.98739946, 20.42272089
sigma_no_job_male_bmi, sigma_no_job_female_bmi = 3.28641337, 2.56429456

df_male = df[df['gender']=='male']
df_female = df[df['gender']=='female']

df_male.loc[df_male.found_job == 1, 'age'] = np.floor(np.random.normal(mu_job_male, sigma_job_male, len(df_male.loc[df_male.found_job == 1])))
df_male.loc[df_male.found_job == 0, 'age'] = np.floor(np.random.normal(mu_no_job_male, sigma_no_job_male, len(df_male.loc[df_male.found_job == 0])))
df_female.loc[df_female.found_job == 1, 'age'] = np.floor(np.random.normal(mu_job_female, sigma_job_female, len(df_female.loc[df_female.found_job == 1])))
df_female.loc[df_female.found_job == 0, 'age'] = np.floor(np.random.normal(mu_no_job_female, sigma_no_job_female, len(df_female.loc[df_female.found_job == 0])))

df = df_male.append(df_female)
df['age'] = df['age'].astype(int)